In [49]:
import pandas as pd

data = pd.read_csv("train_qa.csv").reset_index(drop=True)

In [2]:
data

,paragraph_id,question_id,paragraph,question,answer
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные
5,4857,71431,По мнению Швейцарского банковского объединения...,На что в дальнейшем смогут рассчитывать зарубе...,на предсказуемость швейцарского правового порядка
6,6674,58761,"Судебная власть организована в форме пирамиды,...",Что имеет каждый судебный округ?,суд первой инстанции
7,6980,18725,Сверхкороткие импульсы лазерного излучения исп...,"Что используются и в военных целях, например, ...",Лазеры
8,1154,28808,Разработчики и контролёры‐испытатели проприета...,"Что в гораздо большей степени зависит от того,...",эффективность разработки
9,13109,42131,"Лингвистика языка изучает язык как код, то ест...",Что в лингвистике речи понимается под речевыми...,Статический аспект речи — предмет лингвистики ...


In [50]:
import nltk


nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
def get_answer(text, question, answer):
    sents = nltk.sent_tokenize(text, language="russian")
    word_ans = nltk.word_tokenize(answer, language='russian')
    for sent in sents:
        words = nltk.word_tokenize(sent, language='russian')
        if " ".join(word_ans) in " ".join(words):
            return sent
    return None

In [8]:
print(get_answer(data['paragraph'][0], data['question'][0], data['answer'][0]))

Первые упоминания о строении человеческого тела встречаются в Древнем Египте.


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

def gen_sent_variants(text, question):
    sents = nltk.sent_tokenize(text, language="russian")
    word_quests = nltk.word_tokenize(question, language='russian')
    all_sents = [" ".join(word_quests)]

    for sent in sents:
        words = nltk.word_tokenize(sent, language='russian')
        all_sents.append(" ".join(words))
    tfidf = vectorizer.fit_transform(all_sents)
    cosine = (tfidf * tfidf.T).toarray()
    result = list(range(1, len(all_sents)))
    result = map(lambda t: (sents[t - 1], cosine[0][t]), sorted(result, key=lambda t: cosine[0][t])[-3:])

    return list(result)

In [20]:
gen_sent_variants(data['paragraph'][0], data['question'][0])

[('В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.',
  0.0),
 ('В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека.',
  0.057694724904526734),
 ('Первые упоминания о строении человеческого тела встречаются в Древнем Египте.',
  0.7150191036551836)]

In [21]:
data['question'][0]

'Где встречаются первые упоминания о строении человеческого тела?'

In [49]:
! pip install pymystem3


In [88]:
from pymystem3 import Mystem

m = Mystem()
#    texts = list(map(lambda line: m.lemmatize(line.strip()), f.readlines()))

print(m.analyze("Ай да А.С. Пушкин! Ай да сукин сын!"))

[{'analysis': [{'lex': 'ай', 'wt': 0.9841479957, 'gr': 'INTJ='}], 'text': 'Ай'}, {'text': ' '}, {'analysis': [{'lex': 'да', 'wt': 0.3749187405, 'gr': 'PART='}], 'text': 'да'}, {'text': ' '}, {'analysis': [{'lex': 'а', 'wt': 0.9822148501, 'gr': 'CONJ='}], 'text': 'А'}, {'text': '.'}, {'analysis': [{'lex': 'с', 'wt': 2.2169018090000002e-05, 'gr': 'S,сокр=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)'}], 'text': 'С'}, {'text': '. '}, {'analysis': [{'lex': 'пушкин', 'wt': 0.9968761998000001, 'gr': 'S,фам,муж,од=им,ед'}], 'text': 'Пушкин'}, {'text': '! '}, {'analysis': [{'lex': 'ай', 'wt': 0.9841479957, 'gr': 'INTJ='}], 'text': 'Ай'}, {'text': ' '}, {'analysis': [{'lex': 'да', 'wt': 0.6249755963, 'gr': 'CONJ='}], 'text': 'да'}, {'text': ' '}, {'analysis': [{'lex': 'сукин', 'wt': 1, 'gr': 'A,полн,притяж=(вин,ед,муж,неод|им,ед,муж)'}], 'text': 'сукин'}, {'text': ' '}, {'analysis': [{'lex': 'сын', 'wt': 1, 'gr': 'S,муж,од=им,ед'}], 'text': 'сын'}, {'text':

In [35]:
def get_stat_mystem(phrase):
    analyzed = m.analyze(phrase)
    stats = {'S': 0, 'V': 0, 'A':0, 'PR':0, 'ADV': 0, 'CONJ': 0}
    for t in analyzed:
        if 'analysis' in t and len(t['analysis']) > 0:
            if 'gr' in t['analysis'][0]:
                gr = t['analysis'][0]['gr'].split('=')[0].split(',')[0]
                if gr in stats:
                    stats[gr] += 1
    l = len(analyzed)
    if l == 0:
        l = 1
    return list(map(lambda x: x / l, list(stats.values())))

In [41]:
def gen_features(sent_variants, question, paragraph_id, question_id, answer):
    data_x = []
    data_y = []
    for sent, prob in sent_variants:
        words = nltk.word_tokenize(sent, language='russian')
        for i in range(len(words)):
            for l in range(1, min(11, len(words) - i)):
                variant = words[i:(i + l)]
                tfidf = vectorizer.fit_transform([question, " ".join(words[:i]), " ".join(words[i:(i + l)]), " ".join(words[(i + l):])])
                cosine = (tfidf*tfidf.T).toarray()
                features = [paragraph_id, question_id, prob,
                    i, l, len(words) - i - l, 
                    cosine[0][1], cosine[0][2], cosine[0][3],
                    *get_stat_mystem(" ".join(words[:i])),
                    *get_stat_mystem(" ".join(words[i:(i + l)])),
                    *get_stat_mystem(" ".join(words[(i + l):])),]
                data_x.append(features)
                data_y.append(1 if " ".join(variant) == answer else 0)
    return data_x, data_y


In [58]:
import random

def get_some(train_x, train_y):
    if 1 not in train_y:
        return [], []
    if len(train_x) <= 10:
        return train_x, train_y
    one = train_y.index(1)
    n_train_x = [train_x[one]]
    n_train_y = [train_y[one]]
    magic = 6.0 / (len(train_x) - 1)
    for i in range(len(train_x)):
        if train_y[i] != 1:
            if random.random() < magic:
                n_train_x.append(train_x[i])
                n_train_y.append(train_y[i])
    return n_train_x, n_train_y
    

In [63]:
sent_vars = gen_sent_variants(data['paragraph'][0], data['question'][0])
data_x, data_y = gen_features(sent_vars, data['question'][0], data['paragraph_id'][0], data['question_id'][0], data['answer'][0])
print(get_some(data_x, data_y))

([[14754, 60544, 0.7150191036551836, 7, 3, 1, 0.8880110717198837, 0.0, 0.0, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.0, 0.0, 0.16666666666666666, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [14754, 60544, 0.0, 1, 10, 23, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21052631578947367, 0.0, 0.05263157894736842, 0.05263157894736842, 0.0, 0.0, 0.25, 0.025, 0.1, 0.025, 0.0, 0.05], [14754, 60544, 0.0, 14, 10, 10, 0.0, 0.0, 0.0, 0.2222222222222222, 0.0, 0.037037037037037035, 0.07407407407407407, 0.0, 0.0, 0.25, 0.0625, 0.125, 0.0625, 0.0, 0.0, 0.2222222222222222, 0.0, 0.1111111111111111, 0.0, 0.0, 0.1111111111111111], [14754, 60544, 0.0, 26, 6, 2, 0.0, 0.0, 0.0, 0.2553191489361702, 0.02127659574468085, 0.06382978723404255, 0.06382978723404255, 0.0, 0.0, 0.1, 0.0, 0.2, 0.0, 0.0, 0.2, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0], [14754, 60544, 0.0, 31, 1, 2, 0.0, 0.0, 0.0, 0.23636363636363636, 0.01818181818181818, 0.0

In [102]:
train_data = []
train_y = []


In [103]:
for i in range(826, len(data)):
    if (i + 1) % 100 == 0:
        print(i)
    sent_vars = gen_sent_variants(data['paragraph'][i], data['question'][i])
    data_x, data_y = gen_features(sent_vars, data['question'][i], data['paragraph_id'][i], data['question_id'][i], data['answer'][i])
    data_x, data_y = get_some(data_x, data_y)
    train_data.extend(data_x)
    train_y.extend(data_y)
    if len(train_data) > 100000:
        break;

899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999


KeyboardInterrupt: 

In [104]:
print(len(train_data), len(train_y))

5648 5648


In [105]:
import numpy as np
np_train_data = np.array(train_data)
array = np.zeros((np_train_data.shape[0], np_train_data.shape[1] + 1))
array[:, :-1] = np_train_data
array[:, -1] = np.array(train_y)
label = ['paragraph_id', 'question_id', 'sent_prob',
         'len_l', 'len_a', 'len_r',
         'cos_l', 'cos_a', 'cos_r',
         'l_S', 'l_V', 'l_A', 'l_PR', 'l_ADV', 'l_CONJ',
         'm_S', 'm_V', 'm_A', 'm_PR', 'm_ADV', 'm_CONJ',
         'r_S', 'r_V', 'r_A', 'r_PR', 'r_ADV', 'r_CONJ',
         'result',
         ]
df_features = pd.DataFrame(array, columns=label)
df_features.to_csv("features_saved_1.csv", sep=',')

In [69]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=7, n_estimators=100)

In [70]:
regr.fit(train_data, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [97]:
test_data = pd.read_csv("test_qa_1.csv", sep='\t').reset_index(drop=True)

In [95]:
sent_vars = gen_sent_variants(data['paragraph'][0], data['question'][0])
data_x, data_y = gen_features(sent_vars, data['question'][0], data['paragraph_id'][0], data['question_id'][0], data['answer'][0])


In [96]:
regr.predict(data_x)

array([0.1152982 , 0.04064722, 0.03878327, 0.01024037, 0.00998475,
       0.0078446 , 0.00869975, 0.00869975, 0.00691338, 0.00691338,
       0.03262699, 0.03019002, 0.03634204, 0.00949961, 0.00966352,
       0.00644089, 0.00644089, 0.00492917, 0.0048313 , 0.0048313 ,
       0.06026568, 0.0558052 , 0.05354208, 0.02250967, 0.03250967,
       0.01998543, 0.00568645, 0.0048313 , 0.0048313 , 0.0048313 ,
       0.05925459, 0.05479411, 0.05253099, 0.03149858, 0.02149858,
       0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 ,
       0.03196949, 0.0381953 , 0.03734511, 0.00982126, 0.00876133,
       0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 ,
       0.06369322, 0.06923275, 0.05733727, 0.00901695, 0.00876133,
       0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 ,
       0.07481622, 0.06035574, 0.0274111 , 0.00901695, 0.00876133,
       0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 , 0.0048313 ,
       0.03262699, 0.02863689, 0.02612613, 0.00901695, 0.00876

In [76]:
print(data_x[511])

[14754, 60544, 0.7150191036551836, 7, 3, 1, 0.8880110717198837, 0.0, 0.0, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.0, 0.0, 0.16666666666666666, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [77]:
def gen_features_test(sent_variants, question, paragraph_id, question_id):
    data_x = []
    data_y = []
    variants = []
    for sent, prob in sent_variants:
        words = nltk.word_tokenize(sent, language='russian')
        for i in range(len(words)):
            for l in range(1, min(11, len(words) - i)):
                variant = words[i:(i + l)]
                tfidf = vectorizer.fit_transform([question, " ".join(words[:i]), " ".join(words[i:(i + l)]), " ".join(words[(i + l):])])
                cosine = (tfidf*tfidf.T).toarray()
                features = [paragraph_id, question_id, prob,
                    i, l, len(words) - i - l, 
                    cosine[0][1], cosine[0][2], cosine[0][3],
                    *get_stat_mystem(" ".join(words[:i])),
                    *get_stat_mystem(" ".join(words[i:(i + l)])),
                    *get_stat_mystem(" ".join(words[(i + l):])),]
                data_x.append(features)
                variants.append(variant)
    return data_x, variants

In [79]:
data_x, variants = gen_features_test(sent_vars, data['question'][0], data['paragraph_id'][0], data['question_id'][0])

In [80]:
variants[511]

['в', 'Древнем', 'Египте']

In [100]:
with open('output_1', 'w') as fout:
    for i in range(len(test_data)):
        sent_vars = [gen_sent_variants(test_data['paragraph'][i], test_data['question'][i])[-1]]
        data_x, variants = gen_features_test(sent_vars, test_data['question'][i], test_data['paragraph_id'][i], test_data['question_id'][i])
        predicts = regr.predict(data_x)
        best = np.argmax(predicts)
        if predicts[best] < 0.25:
            print('{}\t{}'.format(test_data['question_id'][i], sent_vars[0][0]), file=fout, flush=True)
            continue
        print('{}\t{}'.format(test_data['question_id'][i], " ".join(variants[best])), file=fout, flush=True)

In [90]:
regr.predict(data_x)

array([0.05931583, 0.05067381, 0.04698244, 0.01741346, 0.01416123,
       0.00845289, 0.00845065, 0.00845065, 0.00838247, 0.00838247,
       0.04801768, 0.04325696, 0.05731912, 0.0164114 , 0.01315917,
       0.00741863, 0.00734821, 0.007455  , 0.00689424, 0.0180571 ,
       0.04362653, 0.06377997, 0.06629301, 0.0164114 , 0.01650007,
       0.01030556, 0.0138291 , 0.007455  , 0.0180571 , 0.00999213,
       0.13246475, 0.15269596, 0.1397221 , 0.04800964, 0.04314105,
       0.03624482, 0.02796106, 0.03288852, 0.02412409, 0.02412409,
       0.24217087, 0.22874902, 0.12324544, 0.05272311, 0.04506881,
       0.02579169, 0.03259762, 0.02381159, 0.02381159, 0.02263512,
       0.07360347, 0.0627759 , 0.0614296 , 0.03707385, 0.02874176,
       0.02074057, 0.01067797, 0.01048454, 0.00930807, 0.00749973,
       0.06314547, 0.06135117, 0.06296647, 0.03675054, 0.0438668 ,
       0.01241506, 0.01048454, 0.01048454, 0.00930807, 0.05848618,
       0.07483459, 0.07600189, 0.08566922, 0.03918055, 0.02966

In [91]:
variants[best]

['именно', 'для', 'аэрофотосъёмки']

In [92]:
test_data.iloc[999]

paragraph_id                                                 4244
question_id                                                 17215
paragraph       Одну из ключевых ролей в совершенствовании фот...
question          Что было впервые использовано в аэрофотосъемке?
Name: 999, dtype: object

In [94]:
test_data.iloc[999]['paragraph']

'Одну из ключевых ролей в совершенствовании фотоаппаратуры сыграло становление аэрофотографии, получившей бурное развитие после Первой мировой войны[17]. Большие скорости полёта требовали коротких выдержек, вынуждая компенсировать их высокой светосилой объективов. При этом, недопустимость геометрических искажений, особенно при фотограмметрии, вынуждала разрабатывать оптику с минимальной дисторсией. Многие конструкции фотозатворов и объективов, привычные в современной фотоаппаратуре, были разработаны специально для аэрофотоаппаратов, лишь потом найдя применение в камерах общего назначения. То же касается вспомогательных механизмов: например, автоматизированная перезарядка фотоаппарата впервые использована именно для аэрофотосъёмки.'

In [101]:
regr.score(train_data, train_y)

0.5284692129843114

In [107]:
sorted(list(zip(regr.feature_importances_, label[:-1])), key=lambda x: x[0])[-10:]

[(0.020130095745681472, 'question_id'),
 (0.02317758014853219, 'cos_r'),
 (0.04765792100275272, 'len_l'),
 (0.054014919790529865, 'cos_l'),
 (0.059964146215300496, 'len_r'),
 (0.06508444741220015, 'r_S'),
 (0.08086332087500375, 'cos_a'),
 (0.09455585699459874, 'len_a'),
 (0.14123741125364014, 'm_V'),
 (0.25529494491258403, 'sent_prob')]